In [1]:
# https://anipose.readthedocs.io/en/latest/aniposelib-tutorial.html

In [2]:
!pip install -U git+https://github.com/abitrolly/aniposelib@bail-on-missing

  Cloning https://github.com/abitrolly/aniposelib (to revision bail-on-missing) to /tmp/pip-req-build-ne5a9gln
  Running command git clone -q https://github.com/abitrolly/aniposelib /tmp/pip-req-build-ne5a9gln
  Running command git checkout -b bail-on-missing --track origin/bail-on-missing
  Switched to a new branch 'bail-on-missing'
  Branch 'bail-on-missing' set up to track remote branch 'bail-on-missing' from 'origin'.


In [3]:
import numpy as np
from aniposelib.boards import CharucoBoard, Checkerboard
from aniposelib.cameras import Camera, CameraGroup
from aniposelib.utils import load_pose2d_fnames

In [4]:
# Calibration (detecting coordinates and probably distortions from each camera)

vidnames = [['calib-charuco-camA-compressed.MOV'],
            ['calib-charuco-camB-compressed.MOV'],
            ['calib-charuco-camC-compressed.MOV']]

cam_names = ['A', 'B', 'C']

n_cams = len(vidnames)

board = CharucoBoard(7, 10,
                     square_length=25, # here, in mm but any unit works
                     marker_length=18.75,
                     marker_bits=4, dict_size=50)


# the videos provided are fisheye, so we need the fisheye option
cgroup = CameraGroup.from_names(cam_names, fisheye=True)

In [5]:
# downloading referenced .MOV files
!gdown --id 1hx7UztHc6bx-keQWWNHYws-SZcRHCyAZ
!gdown --id 1GYCKgIv4uGF9z4vxeVpEbppbzZ-rK6Z1
!gdown --id 1KWB7C9MZT_etNk-35XvR3dbMipyDau4r

Downloading...
From: https://drive.google.com/uc?id=1hx7UztHc6bx-keQWWNHYws-SZcRHCyAZ
To: /content/calib-charuco-camA-compressed.MOV
23.7MB [00:00, 145MB/s]
Downloading...
From: https://drive.google.com/uc?id=1GYCKgIv4uGF9z4vxeVpEbppbzZ-rK6Z1
To: /content/calib-charuco-camB-compressed.MOV
23.3MB [00:00, 143MB/s]
Downloading...
From: https://drive.google.com/uc?id=1KWB7C9MZT_etNk-35XvR3dbMipyDau4r
To: /content/calib-charuco-camC-compressed.MOV
20.5MB [00:00, 114MB/s] 


In [6]:
# this will take about 15 minutes (mostly due to detection)
# it will detect the charuco board in the videos,
# then calibrate the cameras based on the detections, using iterative bundle adjustment
cgroup.calibrate_videos(vidnames, board)

# if you need to save and load
# example saving and loading for later
cgroup.dump('calibration.toml')

calib-charuco-camA-compressed.MOV


100%|█████████████████████████████| 5274/5274 [10:51<00:00,  8.10it/s]


5248 boards detected
calib-charuco-camB-compressed.MOV


100%|█████████████████████████████| 5264/5264 [10:45<00:00,  8.15it/s]


4866 boards detected
calib-charuco-camC-compressed.MOV


100%|█████████████████████████████| 5265/5265 [09:32<00:00,  9.20it/s]


4490 boards detected
defaultdict(<class 'int'>,
            {('A', 'B'): 4425,
             ('A', 'C'): 4078,
             ('B', 'A'): 4425,
             ('B', 'C'): 3386,
             ('C', 'A'): 4078,
             ('C', 'B'): 3386})


/usr/local/lib/python3.7/dist-packages/numba/np/ufunc/parallel.py:355: NumbaWarning: The TBB threading layer requires TBB version 2019.5 or later i.e., TBB_INTERFACE_VERSION >= 11005. Found TBB_INTERFACE_VERSION = 9107. The TBB threading layer is disabled.
  warnings.warn(problem)


error:  13.379718038133845
n_samples: 100
{(0, 1): (2938, array([ 5.61719767, 32.73906066])),
 (0, 2): (2938, array([ 4.96909173, 32.53088562])),
 (1, 2): (2938, array([ 4.73082536, 42.56197745]))}
error: 12.58, mu: 15.0, ratio: 0.572
   Iteration     Total nfev        Cost      Cost reduction    Step norm     Optimality   
       0              1         1.8434e+07                                    4.98e+05    
       1              2         1.1786e+06      1.73e+07       4.75e+04       3.77e+04    
       2              3         3.3980e+04      1.14e+06       1.32e+04       5.80e+04    
       3              5         2.4309e+04      9.67e+03       3.81e+03       4.48e+04    
       4              7         1.6528e+04      7.78e+03       8.29e+02       8.39e+03    
       5              8         1.0971e+04      5.56e+03       1.16e+03       2.71e+04    
       6              9         8.1094e+03      2.86e+03       1.37e+03       3.34e+04    
       7             10         7.844

In [7]:
import cv2; cv2.__version__

'3.4.15'

In [8]:
!pip list tbb

Package                       Version
----------------------------- --------------
absl-py                       0.12.0
alabaster                     0.7.12
albumentations                0.1.12
altair                        4.1.0
aniposelib                    0.4.1
appdirs                       1.4.4
argcomplete                   1.12.3
argon2-cffi                   20.1.0
arviz                         0.11.2
astor                         0.8.1
astropy                       4.3.1
astunparse                    1.6.3
atari-py                      0.2.9
atomicwrites                  1.4.0
attrs                         21.2.0
audioread                     2.1.9
autograd                      1.3
Babel                         2.9.1
backcall                      0.2.0
beautifulsoup4                4.6.3
bleach                        4.0.0
blis                          0.4.1
bokeh                         2.3.3
Bottleneck                    1.3.2
branca                        0.4.2
bs4         